Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Fthree-failed-saves-195436%2Fthree_failed_saves.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/three-failed-saves-195436)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“What is the chance of failing all three saves of the spell Flesh to Stone?”](https://rpg.stackexchange.com/a/195452/71245)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce, tabulate
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.4.0", "tabulate"]
        try:
            import piplite ; await piplite.install(requirements)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce, tabulate

In [2]:
from dyce import H
from dyce.evaluation import HResult, foreach
from enum import IntEnum
from functools import partial
import pandas

d20 = H(20)

class SavesResult(IntEnum):
    STONE = False
    FLESH = True

distributions_by_target = {}

for target in range(20, 1, -1):
    def count_additional_failures_after_first_roll_fails(result: HResult):
        if result.outcome < target:
            # We failed the first saving throw. Now we want to compute the likelihood of
            # getting at least two more failures over the next four rolls. (If we didn't
            # get at least two, it means we had three successes, and we're done.)

            # We start with computing the likelihood of getting a failure in a single
            # roll.
            expected_failures_for_one_roll = result.h.lt(target)  # E.g., H({False: 5, True: 15}) for target 16

            # Then, we can leverage a computational trick to very efficiently compute
            # the number of expected failures in four rolls. The outcomes are the number
            # of the expected failures, and the counts are how often one can expect to
            # get that precise number of failures. For example, at target 16, this value
            # is H({0: 625, 1: 7500, 2: 33750, 3: 67500, 4: 50625}). That means out of
            # 160,000 possible rolls, we can expect zero failures 625 times, one failure
            # 7,500 times, two failures 33,750 times, three failures 67,500, and four
            # failures 50,625 times.
            expected_number_of_failures_in_four_rolls = 4@expected_failures_for_one_roll

            # Finally, we count how often we can expect to experience at least two
            # failures among those four rolls. If we get at least two, combined with the
            # the first, it means we've missed three saves, and we're petrified.
            expectation_of_at_least_two_failures_in_four_rolls = expected_number_of_failures_in_four_rolls.ge(2)  # E.g., H({False: 8125, True: 151875}) for target 16

            return expectation_of_at_least_two_failures_in_four_rolls
        else:
            # We made the first saving throw, so we treat that as terminal and return 0
            # failures.
            return 0

    # Set it all in motion!
    expectation_of_final_failure = foreach(
        count_additional_failures_after_first_roll_fails,
        d20,
    )  # E.g., H({False: 8125, True: 151875}) for target 16

    # Note that our current histogram tells us how often we can expect to *fail* the
    # save. To understand how often we expect to succeed, we have to negate what we have.
    expectation_of_final_success = expectation_of_final_failure.ne(True)  # E.g., H({False: 151875, True: 8125}) for target 16

    # Now we can translate our raw results to our enum.
    distributions_by_target[target] = H(
        (SavesResult(outcome), count) for outcome, count in expectation_of_final_success.items()
    )  # E.g., H({<SavesResult.STONE: 0>: 151875, <SavesResult.FLESH: 1>: 8125}) for target 16

    # We could have written most of the above very compactly:
    compact_raw = foreach(
        lambda result: (4@result.h.lt(target)).ge(2) if result.outcome < target else 0,
        d20,
    ).ne(True)
    assert compact_raw == distributions_by_target[target]

    # We also could have also counted successes from the outset instead of converting
    # from failures.
    compact_raw = foreach(
        lambda result: 1 if result.outcome >= target else (4@result.h.ge(target)).ge(3),  # we need 3 or more successes to make it out of the woods
        d20,
    )
    assert compact_raw == distributions_by_target[target]

COLUMNS = ["Expectation of Petrification", "Expectation of Life as Usual"]
df = pandas.DataFrame(columns=COLUMNS)

for target, h in distributions_by_target.items():
    row = pandas.DataFrame([[
        f"{h.get(SavesResult.STONE, 0) / h.total:.2%}",
        f"{h.get(SavesResult.FLESH, 0) / h.total:.2%}",
    ]], columns=COLUMNS, index=[target])
    df = pandas.concat((df, row))

df.index.name = "Target"
display(df)  # raw: df.to_markdown()

,Expectation of Petrification,Expectation of Life as Usual
Target,,
20,94.95%,5.05%
19,89.67%,10.33%
18,83.98%,16.02%
17,77.82%,22.18%
16,71.19%,28.81%
15,64.14%,35.86%
14,56.78%,43.22%
13,49.25%,50.75%
12,41.72%,58.28%


In [3]:
from anydyce import jupyter_visualize

jupyter_visualize(
    (
        (f"Expectation of turning\nto stone at target {target}", h)
        for target, h in distributions_by_target.items()
    ),
    initial_alpha=0.5,
    initial_burst_cmap_inner="Set1",
)